In [1]:
import __init__

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

from db.pgdf import PgDf
from db.postgresql import PostgreSqlDB

d = PostgreSqlDB()
pg = PgDf()

df = pg.read_sql("""WITH t AS (
SELECT t.codename, t.side,
t.profit,
CASE WHEN t.result = 'lose' THEN -1
    WHEN t.result = 'win' THEN 1
    ELSE 0
END AS result,
CASE WHEN (mado > 2) THEN 1 ELSE 0 END AS mado,
CASE WHEN abs(trend_rate) > 0.75 THEN 1 ELSE 0 END AS trend,
CASE WHEN len_std > 1.5 THEN 1 ELSE 0 END AS long_candle,
CASE WHEN (momiai >= 0.4) THEN 1 ELSE 0 END AS momiai,
peak_broken
FROM zz_strtg_params as z
JOIN trades as t ON t.order_id = z.order_id
WHERE t.trade_name = 'zzstrat_top3000vol'
AND abs(t.profit) > 0
AND t.open_price > 1000
)

SELECT COUNT(*) as cnt, side, result,mado,trend,long_candle,momiai, peak_broken, (SUM(t.profit)/20000) as profit
FROM t
-- WHERE momiai = 0
GROUP BY side, result, mado,trend,long_candle, momiai, peak_broken
ORDER BY 2,4,5,6,7,8,3
;""")
df

ERROR:root:(psycopg2.errors.UndefinedColumn) column "momiai" does not exist
LINE 11: CASE WHEN (momiai >= 0.4) THEN 1 ELSE 0 END AS momiai,
                    ^
HINT:  Perhaps you meant to reference the column "z.momiai1" or the column "z.momiai2".

[SQL: WITH t AS (
SELECT t.codename, t.side,
t.profit,
CASE WHEN t.result = 'lose' THEN -1
    WHEN t.result = 'win' THEN 1
    ELSE 0
END AS result,
CASE WHEN (mado > 2) THEN 1 ELSE 0 END AS mado,
CASE WHEN abs(trend_rate) > 0.75 THEN 1 ELSE 0 END AS trend,
CASE WHEN len_std > 1.5 THEN 1 ELSE 0 END AS long_candle,
CASE WHEN (momiai >= 0.4) THEN 1 ELSE 0 END AS momiai,
peak_broken
FROM zz_strtg_params as z
JOIN trades as t ON t.order_id = z.order_id
WHERE t.trade_name = 'zzstrat_top3000vol'
AND abs(t.profit) > 0
AND t.open_price > 1000
)

SELECT COUNT(*) as cnt, side, result,mado,trend,long_candle,momiai, peak_broken, (SUM(t.profit)/20000) as profit
FROM t
-- WHERE momiai = 0
GROUP BY side, result, mado,trend,long_candle, momiai, peak_brok

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "momiai" does not exist
LINE 11: CASE WHEN (momiai >= 0.4) THEN 1 ELSE 0 END AS momiai,
                    ^
HINT:  Perhaps you meant to reference the column "z.momiai1" or the column "z.momiai2".

[SQL: WITH t AS (
SELECT t.codename, t.side,
t.profit,
CASE WHEN t.result = 'lose' THEN -1
    WHEN t.result = 'win' THEN 1
    ELSE 0
END AS result,
CASE WHEN (mado > 2) THEN 1 ELSE 0 END AS mado,
CASE WHEN abs(trend_rate) > 0.75 THEN 1 ELSE 0 END AS trend,
CASE WHEN len_std > 1.5 THEN 1 ELSE 0 END AS long_candle,
CASE WHEN (momiai >= 0.4) THEN 1 ELSE 0 END AS momiai,
peak_broken
FROM zz_strtg_params as z
JOIN trades as t ON t.order_id = z.order_id
WHERE t.trade_name = 'zzstrat_top3000vol'
AND abs(t.profit) > 0
AND t.open_price > 1000
)

SELECT COUNT(*) as cnt, side, result,mado,trend,long_candle,momiai, peak_broken, (SUM(t.profit)/20000) as profit
FROM t
-- WHERE momiai = 0
GROUP BY side, result, mado,trend,long_candle, momiai, pe

In [12]:
print("momiai=1: %d" % (len(df[df.momiai == 1])))
print("total   : %d" % (len(df)))

momiai=1: 56
total   : 102


In [10]:
df = pg.read_sql("""WITH t AS (
SELECT t.codename,
t.side,
t.profit,
CASE WHEN t.result = 'lose' THEN -1
    WHEN t.result = 'win' THEN 1
    ELSE 0
END AS result,
CASE WHEN (mado > 1) THEN 1 ELSE 0 END AS mado,
CASE WHEN (abs(trend_rate) > 0.7) THEN 1 ELSE 0 END AS trend,
CASE WHEN ABS(len_std) > 1.5 THEN 1 ELSE 0 END AS long_candle,
prefer_recent_peaks,
CASE WHEN (reversed_rate > 0.3) THEN 1 ELSE 0 END AS reversed,
CASE WHEN (ABS(acc) > 0.6) THEN 1 ELSE 0 END AS acc,
CASE WHEN (momiai >= 0.5) THEN 1 ELSE 0 END AS momiai,
peak_broken
FROM zz_strtg_params as z
JOIN trades as t ON t.order_id = z.order_id
WHERE t.trade_name = 'zzstrat_top1000vol_bothside'
AND abs(t.profit) > 0
AND t.open_price > 1000
), g AS (
SELECT codename, side, COUNT(*) as cnt, result,mado,trend,long_candle,reversed,momiai, peak_broken, (SUM(t.profit)/20000) as profit
FROM t
GROUP BY codename, side, result, mado,trend,long_candle,reversed, momiai, peak_broken
), w AS (SELECT * FROM g WHERE result = 1),
l AS (SELECT * FROM g WHERE result = -1)

SELECT w.codename, w.side, w.cnt, l.cnt, w.mado, w.trend, w.long_candle, w.reversed, w.momiai, w.peak_broken
FROM w LEFT JOIN l on w.codename = l.codename and w.mado = l.mado and w.trend = l.trend and
w.long_candle = l.long_candle and w.reversed = l.reversed and w.momiai = l.momiai and w.peak_broken = l.peak_broken
-- WHERE w.cnt > 2
ORDER BY w.codename, w.cnt desc
;""")
df

,codename,side,cnt,cnt,mado,trend,long_candle,reversed,momiai,peak_broken
0,1306.T,-1,1,NaN,0,1,0,1,0,0
1,1306.T,1,1,1.0,0,1,0,0,1,0
2,1306.T,-1,1,1.0,0,1,0,0,1,0
3,1306.T,-1,1,NaN,0,1,0,1,1,0
4,1306.T,-1,1,NaN,1,1,0,0,0,0
5,1306.T,-1,1,NaN,1,1,0,1,1,0
6,1357.T,1,1,NaN,0,0,0,1,1,0
7,1357.T,1,1,NaN,0,1,0,0,1,0
8,1360.T,-1,1,NaN,0,1,0,0,0,0
9,1360.T,1,1,NaN,0,1,0,0,1,0
